In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import numpy as np
import pandas as pd
import tensorflow as tf    

In [3]:
TRAIN_DATA_URL="https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL="https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path=tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path=tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [4]:
np.set_printoptions(precision=3, suppress=True)

In [7]:
df = pd.read_csv(train_file_path)

In [8]:
df.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [9]:
df.dtypes

survived                int64
sex                    object
age                   float64
n_siblings_spouses      int64
parch                   int64
fare                  float64
class                  object
deck                   object
embark_town            object
alone                  object
dtype: object

In [10]:
LABEL_COLUMN="survived"

In [11]:
LABELS=[0,1]

In [12]:
def get_dataset(filepath, **kwargs):
    dataset=tf.data.experimental.make_csv_dataset(
        filepath,
        batch_size=5,
        label_name=LABEL_COLUMN,
        na_value="?",
        num_epochs=1,
        ignore_errors=True,
        **kwargs 
    )
    return dataset
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [13]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key, value.numpy()))

In [14]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'male']
age                 : [23. 35.  1. 28. 29.]
n_siblings_spouses  : [0 1 4 0 1]
parch               : [0 0 1 0 0]
fare                : [ 7.854 83.475 39.688  8.663 21.   ]
class               : [b'Third' b'First' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'C' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'n' b'y' b'n']


In [15]:
CSV_COLUMNS = ['survived','sex','age','n_siblings_spouse','parch','fare','class','deck','embark_town','alone']
temp_dataset=get_dataset(train_file_path, column_names=CSV_COLUMNS)
show_batch(temp_dataset)

sex                 : [b'male' b'female' b'female' b'female' b'female']
age                 : [40. 32. 26. 25. 45.]
n_siblings_spouse   : [1 1 1 1 1]
parch               : [4 1 1 1 1]
fare                : [ 27.9    15.5    26.     30.    164.867]
class               : [b'Third' b'Third' b'Second' b'Second' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Queenstown' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'n' b'n' b'n']


In [16]:
SELECT_COLUMNS = ['survived','age','n_siblings_spouses','class','deck','alone']
temp_dataset=get_dataset(train_file_path, select_columns=SELECT_COLUMNS)
show_batch(temp_dataset)

age                 : [28. 37. 44. 28. 28.]
n_siblings_spouses  : [1 0 0 0 0]
class               : [b'Third' b'Third' b'First' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'B' b'unknown' b'unknown']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [17]:
SELECT_COLUMNS = ['survived','age','n_siblings_spouses','parch','fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path,
                          select_columns=SELECT_COLUMNS,
                          column_defaults=DEFAULTS)
show_batch(temp_dataset)

age                 : [29. 28. 18. 48. 21.]
n_siblings_spouses  : [0. 0. 1. 1. 0.]
parch               : [0. 0. 1. 2. 0.]
fare                : [211.337   7.75   20.212  65.      7.775]


In [21]:
example_batch, labels_batch = next(iter(temp_dataset))

In [19]:
def pack(features, label):
    return tf.stack(list(features.values()),axis=-1),label

In [23]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[ 29.      0.      0.    211.337]
 [ 28.      0.      0.      7.75 ]
 [ 18.      1.      1.     20.212]
 [ 48.      1.      2.     65.   ]
 [ 21.      0.      0.      7.775]]

[1 1 0 1 0]


In [24]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'male']
age                 : [23. 35.  1. 28. 29.]
n_siblings_spouses  : [0 1 4 0 1]
parch               : [0 0 1 0 0]
fare                : [ 7.854 83.475 39.688  8.663 21.   ]
class               : [b'Third' b'First' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'C' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'n' b'y' b'n']


In [25]:
example_batch, labels_batch = next(iter(temp_dataset))

In [28]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names
    
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        
        return features, labels

In [29]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch','fare']

packed_train_data = raw_train_data.map(
            PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
            PackNumericFeatures(NUMERIC_FEATURES))


In [30]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'male']
class               : [b'Third' b'First' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'C' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'n' b'y' b'n']
numeric             : [[23.     0.     0.     7.854]
 [35.     1.     0.    83.475]
 [ 1.     4.     1.    39.688]
 [28.     0.     0.     8.663]
 [29.     1.     0.    21.   ]]


In [31]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [32]:
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [33]:
MEAN = np.array(desc.T['mean'])
STD =  np.array(desc.T['std'])

In [34]:
def normalize_numeric_data(data,mean,std):
    return (data-mean)/std

In [35]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column


NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000001ACACEC0840>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [36]:
example_batch['numeric']

<tf.Tensor: id=599, shape=(5, 4), dtype=float32, numpy=
array([[23.   ,  0.   ,  0.   ,  7.854],
       [35.   ,  1.   ,  0.   , 83.475],
       [ 1.   ,  4.   ,  1.   , 39.688],
       [28.   ,  0.   ,  0.   ,  8.663],
       [29.   ,  1.   ,  0.   , 21.   ]], dtype=float32)>

In [37]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

W0826 11:16:11.318313 12712 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\feature_column\feature_column_v2.py:2758: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


array([[-0.53 , -0.474, -0.479, -0.486],
       [ 0.429,  0.395, -0.479,  0.899],
       [-2.288,  3.001,  0.782,  0.097],
       [-0.13 , -0.474, -0.479, -0.471],
       [-0.05 ,  0.395, -0.479, -0.245]], dtype=float32)

In [38]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [39]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
               key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [40]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [41]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

W0826 11:20:30.234798 12712 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\lookup_ops.py:1347: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0826 11:20:30.287794 12712 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\feature_column\feature_column_v2.py:4307: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0826 11:20:30.289807 12712 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\feature_column\feature_column_v2.py:4362: VocabularyListCategoricalColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be 

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [43]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [44]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.53  -0.474
 -0.479 -0.486  1.     0.   ]


In [45]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'), 
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [46]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [47]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 9s 71ms/step - loss: 0.5103 - accuracy: 0.6889
Epoch 2/20
126/126 [==============================] - 5s 37ms/step - loss: 0.4246 - accuracy: 0.8262
Epoch 3/20
126/126 [==============================] - 5s 38ms/step - loss: 0.4061 - accuracy: 0.8367
Epoch 4/20
126/126 [==============================] - 5s 38ms/step - loss: 0.3940 - accuracy: 0.8496
Epoch 5/20
126/126 [==============================] - 5s 38ms/step - loss: 0.3847 - accuracy: 0.8509
Epoch 6/20
126/126 [==============================] - 5s 38ms/step - loss: 0.3760 - accuracy: 0.8567
Epoch 7/20
126/126 [==============================] - 5s 39ms/step - loss: 0.3687 - accuracy: 0.8505
Epoch 8/20
126/126 [==============================] - 5s 38ms/step - loss: 0.3624 - accuracy: 0.8523
Epoch 9/20
126/126 [==============================] - 5s 38ms/step - loss: 0.3556 - accuracy: 0.8581
Epoch 10/20
126/126 [==============================] - 5s 37ms/step - loss: 0.3495 - accura

In [48]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

     53/Unknown - 2s 32ms/step - loss: 0.5189 - accuracy: 0.8030

Test Loss 0.5189008348674145, Test Accuracy 0.8030303120613098


In [50]:
predictions = model.predict(test_data)

for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    print("Predicted survival: {:.2%}".format(prediction[0]),
         " | Actual outcome: ",
         ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 99.99%  | Actual outcome:  SURVIVED
Predicted survival: 0.57%  | Actual outcome:  SURVIVED
Predicted survival: 1.15%  | Actual outcome:  DIED
Predicted survival: 11.70%  | Actual outcome:  DIED
Predicted survival: 48.03%  | Actual outcome:  SURVIVED
